In [1]:
import findspark

findspark.init('/usr/local/spark')

In [ ]:
# check some of the env variables
!env | grep -i spark

In [2]:
from pyspark.sql import SparkSession
# from pyspark.sql import SQLContext

In [4]:
spark = SparkSession.builder \
        .master("spark://10.67.22.59:7077") \
        .appName("3rd") \
        .config('spark.jars.packages',
                'org.diana-hep:spark-root_2.11:0.1.13,org.diana-hep:histogrammar-sparksql_2.11:1.0.4,org.apache.hadoop:hadoop-aws:2.7.0',)\
        .config('spark.cores.max',3)\
        .getOrCreate()

spark

In [5]:
# get spark context -> entry point used to work with RDD
sc = spark.sparkContext
sc

<SparkContext master=spark://10.67.22.59:7077 appName=3rd>

In [6]:
# python dataset
data = [1,2,3,4,5,6,7,8]

# parallelize
sc.parallelize(data).count()

8

In [ ]:
#verifying spark
import random

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

num_samples = 1000000
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4.0 * count / num_samples
print(pi)

In [8]:
buck = 'https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/LCPmodB-Y3_CMS_FCNC/'

local = '/data/FCNC/'

tofile = 'SingleMuon_Run2016B_ver1-Nano1June2019_ver1-v1/4F32E8E2-3E72-E84D-A12B-CA4CB942D65C_Skim.root'

In [ ]:
import os

df = spark.read.format("org.dianahep.sparkroot").load('hdfs:///root' + tofile)

print(df.count())

In [ ]:
sqlContext = SQLContext(sc)

sqlContext.read.format("org.dianahep.sparkroot").option("tree", "Events")\
.load("file:"+tofile)

In [ ]:
!$SPARK_HOME/bin/pyspark --master spark://ip-10.67.22.59:7077 --packages org.apache.hadoop:hadoop-aws:2.7.0

In [9]:
#Read the ROOT file into a Spark DataFrame
df = spark.read\
    .format("org.dianahep.sparkroot")\
    .load("s3://" + buck + tofile)
# ... and print the number of events
# print df.count()

Py4JJavaError: An error occurred while calling o68.load.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.dianahep.sparkroot.package$RootTableScan.<init>(sparkroot.scala:81)
	at org.dianahep.sparkroot.DefaultSource.createRelation(sparkroot.scala:146)
	at org.dianahep.sparkroot.DefaultSource.createRelation(sparkroot.scala:143)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:318)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:223)
	

In [ ]:
sc.stop()